<a href="https://colab.research.google.com/github/miramnair/sentiment_analysis_BERT/blob/main/Sentiment_analysis_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install dependencies**

In [ ]:
!pip3 install torch torchvision torchaudio

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup


**Instantiate the Model**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

**Caluclate sentiment on a sample string**


***Very Negative (Score: 1)***
Indicates strong negative sentiment. The review expresses a high level of dissatisfaction or negative experience.

***Negative (Score: 2)***
Indicates a negative sentiment. The review reflects dissatisfaction or an unpleasant experience but is not as strongly negative as a score of 1.

***Neutral (Score: 3)***
Indicates a neutral sentiment. The review does not lean strongly towards either positive or negative. It might reflect an average or mixed experience.

***Positive (Score: 4)***
Indicates a positive sentiment. The review expresses satisfaction or a good experience, though it is not as strongly positive as a score of 5.

***Very Positive (Score: 5)***
Indicates strong positive sentiment. The review expresses a high level of satisfaction or an excellent experience.

In [ ]:
tokens = tokenizer.encode('The customer service was average.', return_tensors='pt')

In [ ]:
result = model(tokens)

In [ ]:
result.logits

tensor([[-0.2740,  1.4251,  2.1138, -0.3489, -2.3076]],
       grad_fn=<AddmmBackward0>)

In [ ]:
#+ 1: BERT models typically use zero-based indexing for classes, so adding 1 adjusts the index to match the 1-5 sentiment scale.
int(torch.argmax(result.logits))+1

3

**Collect Reviews from website and calculate sentiment score**

In [ ]:
def get_reviews_from_page(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    regex = re.compile('.*reviewContent.*')
    results = soup.find_all('div', {'class': regex})
    reviews = [result.text.strip() for result in results]
    return reviews

base_url = 'https://uk.trustpilot.com/review/www.wessexwater.co.uk'
reviews = []

# Extract reviews of first 10 pages
for page in range(1, 10):
    url = f"{base_url}?page={page}"
    page_reviews = get_reviews_from_page(url)
    reviews.extend(page_reviews)

In [ ]:
len(reviews)

180

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])
df.head()

,review
0,Fantastic Customer ServiceI would like to say ...
1,Pitch fibre issues Ryan called before turning ...
2,"Problems solved, rapidly! Noticed waste water ..."
3,Wessex Water - brilliant service!Recently cont...
4,we have some significant sewerage…we have some...


In [ ]:

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
#BERT models have a maximum input length of 512 tokens
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
# Filter out bad reviews (sentiment score 1 or 2)
bad_reviews = df[df['sentiment'] <= 2]

# Display bad reviews
print("Bad Reviews:")
print(bad_reviews)

Bad Reviews:
                                                review  sentiment
1    Pitch fibre issues Ryan called before turning ...          2
2    Problems solved, rapidly! Noticed waste water ...          1
11   Disrespectful and RudeAttempted to unlock our ...          1
16   In the field behind our property there…In the ...          1
17   sewerage flooding in a public carparkReported ...          1
23   Timely response to my email queryI emailed wit...          1
24   Blocked InterceptorJob number 62354825001Conta...          1
27   5* to Danny HillHuge shout out to Danny for hi...          1
28   Blocked public sewer The Wessex water came to ...          1
29   Customer comes after profitCustomer comes afte...          1
39   Burst water main feed to houseI reported a bur...          1
41   Leak within boundary. The leak was assessed by...          2
49   Help when we needed itThe domestic water main ...          2
52   Very politeVery polite, considerate and courte...         